# CH09-02: 技術選型決策樹

**課程目標:**
- 掌握不同 NLP 任務的技術選型決策流程
- 理解模型大小與效能權衡
- 學會根據實際需求選擇最合適的技術方案
- 建立系統化的技術決策框架

**學習時間:** 約 90 分鐘

**前置知識:**
- 已完成 CH09-01 技術體系回顧
- 熟悉各類 NLP 技術與模型

---

## 📚 目錄

1. [技術選型核心原則](#1)
2. [文本分類任務選型決策樹](#2)
3. [命名實體識別 (NER) 選型決策樹](#3)
4. [文本生成任務選型決策樹](#4)
5. [問答系統選型決策樹](#5)
6. [模型大小與效能權衡](#6)
7. [實際案例選型分析](#7)
8. [決策樹實戰工具](#8)

---

In [ ]:
# 環境設定與套件導入
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, display, Markdown
import warnings
warnings.filterwarnings('ignore')

# 設定中文顯示
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 設定顯示風格
sns.set_style('whitegrid')
sns.set_palette('Set2')

print("✅ 環境設定完成")
print(f"NumPy 版本: {np.__version__}")
print(f"Pandas 版本: {pd.__version__}")

<a id="1"></a>
## 1. 技術選型核心原則

### 1.1 選型決策的六大維度

在選擇 NLP 技術方案時,需要綜合考慮以下維度:

#### 1. 任務類型 (Task Type)
- **理解型任務:** 分類、NER、關係抽取、情感分析
- **生成型任務:** 機器翻譯、摘要、對話、創作
- **混合型任務:** 問答系統、文本改寫

#### 2. 數據規模 (Data Scale)
- **極少 (<1K):** 零樣本學習、少樣本學習
- **少量 (1K-10K):** 預訓練模型微調
- **中等 (10K-100K):** 預訓練模型微調或輕量級訓練
- **大量 (>100K):** 從頭訓練或大規模微調

#### 3. 計算資源 (Computing Resources)
- **低算力 (CPU only):** 傳統 ML、小型模型
- **中等算力 (單 GPU):** BERT Base、小型 Transformer
- **高算力 (多 GPU/TPU):** BERT Large、GPT 系列

#### 4. 效能要求 (Performance Requirements)
- **基礎要求:** 傳統 ML、小型預訓練模型
- **高效能要求:** 大型預訓練模型、集成方法
- **SOTA 要求:** 最新 LLM、精細調優

#### 5. 延遲限制 (Latency Constraints)
- **實時 (<100ms):** 模型壓縮、量化、蒸餾
- **近實時 (<1s):** 中型模型、優化推理
- **批處理 (>1s):** 大型模型、高精度優先

#### 6. 可解釋性需求 (Interpretability)
- **高可解釋性:** 傳統 ML、規則系統
- **中等可解釋性:** Attention 視覺化、特徵重要性
- **低可解釋性:** 端到端深度學習

---

In [ ]:
# 視覺化技術選型六大維度
dimensions = ['任務類型', '數據規模', '計算資源', '效能要求', '延遲限制', '可解釋性']

# 不同方案在各維度的評分 (1-10)
traditional_ml = [5, 3, 10, 5, 10, 9]
bert_base = [8, 6, 6, 8, 6, 5]
bert_large = [9, 7, 4, 9, 4, 5]
gpt_small = [9, 5, 5, 9, 5, 3]
gpt_large = [10, 8, 2, 10, 2, 2]

# 繪製雷達圖
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='polar')

num_vars = len(dimensions)
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]

# 繪製各方案
solutions = [
    ('傳統 ML', traditional_ml, 'blue'),
    ('BERT Base', bert_base, 'green'),
    ('BERT Large', bert_large, 'orange'),
    ('GPT (小型)', gpt_small, 'purple'),
    ('GPT (大型)', gpt_large, 'red')
]

for name, values, color in solutions:
    values_plot = values + values[:1]
    ax.plot(angles, values_plot, 'o-', linewidth=2, label=name, color=color, markersize=6)
    ax.fill(angles, values_plot, alpha=0.15, color=color)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(dimensions, fontsize=12)
ax.set_ylim(0, 10)
ax.set_yticks([2, 4, 6, 8, 10])
ax.set_yticklabels(['2', '4', '6', '8', '10'], fontsize=10)
ax.set_title('不同技術方案在六大維度的表現對比', fontsize=16, fontweight='bold', pad=30)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.15), fontsize=11)
ax.grid(True)

plt.tight_layout()
plt.show()

print("\n💡 選型洞察:")
print("  1. 傳統 ML: 資源友好,延遲低,可解釋性強,但效能有限")
print("  2. BERT Base: 平衡方案,適合大多數任務")
print("  3. BERT Large: 高效能,但資源需求高")
print("  4. GPT 小型: 生成能力強,資源適中")
print("  5. GPT 大型: 最強效能,但資源與延遲是瓶頸")
print("\n🎯 選型建議: 根據實際約束條件選擇最平衡的方案")

### 1.2 技術選型決策流程

通用的技術選型決策流程:

In [ ]:
# 使用 Mermaid 語法繪製通用決策流程
mermaid_general_flow = '''
graph TD
    Start[開始技術選型] --> Q1{任務類型?}
    
    Q1 -->|理解型| Understanding[文本分類/NER/關係抽取]
    Q1 -->|生成型| Generation[翻譯/摘要/對話]
    Q1 -->|混合型| Hybrid[問答/改寫]
    
    Understanding --> Q2{數據量?}
    Generation --> Q2
    Hybrid --> Q2
    
    Q2 -->|<1K| ZeroFew[零樣本/少樣本學習]
    Q2 -->|1K-10K| SmallData[小數據微調]
    Q2 -->|>10K| LargeData[充足數據]
    
    ZeroFew --> Q3{計算資源?}
    SmallData --> Q3
    LargeData --> Q3
    
    Q3 -->|CPU only| LowCompute[傳統ML/小型模型]
    Q3 -->|單GPU| MidCompute[BERT Base/GPT小型]
    Q3 -->|多GPU| HighCompute[BERT Large/GPT大型]
    
    LowCompute --> Q4{延遲要求?}
    MidCompute --> Q4
    HighCompute --> Q4
    
    Q4 -->|實時<100ms| Realtime[模型壓縮/量化]
    Q4 -->|近實時<1s| NearRealtime[優化推理]
    Q4 -->|批處理>1s| Batch[高精度優先]
    
    Realtime --> Final[最終技術方案]
    NearRealtime --> Final
    Batch --> Final
    
    style Start fill:#FFD700,stroke:#333,stroke-width:3px
    style Final fill:#4CAF50,stroke:#333,stroke-width:3px
    style Q1 fill:#87CEEB,stroke:#333,stroke-width:2px
    style Q2 fill:#87CEEB,stroke:#333,stroke-width:2px
    style Q3 fill:#87CEEB,stroke:#333,stroke-width:2px
    style Q4 fill:#87CEEB,stroke:#333,stroke-width:2px
'''

display(HTML("""
<div style="border:3px solid #4CAF50; padding:20px; border-radius:15px; background-color:#F1F8E9;">
    <h3 style="color:#2E7D32;">🌳 通用技術選型決策流程</h3>
    <p><strong>四步決策法:</strong></p>
    <ol style="line-height:1.8;">
        <li><strong>步驟 1:</strong> 確定任務類型 (理解/生成/混合)</li>
        <li><strong>步驟 2:</strong> 評估數據規模 (少/中/多)</li>
        <li><strong>步驟 3:</strong> 考慮計算資源 (CPU/單GPU/多GPU)</li>
        <li><strong>步驟 4:</strong> 確定延遲要求 (實時/近實時/批處理)</li>
    </ol>
    <p style="color:#666; font-style:italic; margin-top:15px;">💡 提示: 在 Markdown 編輯器中使用上方 Mermaid 代碼可視覺化完整決策樹</p>
</div>
"""))

print("\n🔍 決策要點:")
print("  1. 任務類型決定模型架構 (Encoder/Decoder/Encoder-Decoder)")
print("  2. 數據規模決定訓練策略 (零樣本/微調/從頭訓練)")
print("  3. 計算資源決定模型大小")
print("  4. 延遲要求決定優化策略")

<a id="2"></a>
## 2. 文本分類任務選型決策樹

### 2.1 文本分類任務特點

**任務定義:** 將文本分配到預定義的類別

**常見應用:**
- 情感分析 (正面/負面/中性)
- 主題分類 (新聞分類、內容審核)
- 垃圾郵件檢測
- 意圖識別

**評估指標:**
- Accuracy, Precision, Recall, F1-Score
- Confusion Matrix
- ROC-AUC (二分類)

---

In [ ]:
# 文本分類決策樹
text_classification_tree = '''
graph TD
    Start[文本分類任務] --> Q1{數據量?}
    
    Q1 -->|<500 樣本| Few[極少數據]
    Q1 -->|500-5K| Small[少量數據]
    Q1 -->|5K-50K| Medium[中等數據]
    Q1 -->|>50K| Large[充足數據]
    
    Few --> F1[零樣本學習<br/>GPT-3/4 Prompt]
    Few --> F2[少樣本學習<br/>Few-shot Prompt]
    
    Small --> S1{類別數?}
    S1 -->|2-5 類| S1A[BERT Base 微調<br/>DistilBERT]
    S1 -->|>5 類| S1B[數據增強<br/>+ BERT 微調]
    
    Medium --> M1{效能要求?}
    M1 -->|基礎| M1A[TF-IDF + LR<br/>FastText]
    M1 -->|高| M1B[BERT Base 微調<br/>RoBERTa]
    
    Large --> L1{計算資源?}
    L1 -->|有限| L1A[BERT Base<br/>DistilBERT]
    L1 -->|充足| L1B[BERT Large<br/>RoBERTa Large]
    L1 -->|極高| L1C[GPT-3 微調<br/>或從頭訓練]
    
    style Start fill:#FFD700,stroke:#333,stroke-width:3px
    style F1 fill:#90EE90,stroke:#333,stroke-width:2px
    style F2 fill:#90EE90,stroke:#333,stroke-width:2px
    style S1A fill:#87CEEB,stroke:#333,stroke-width:2px
    style S1B fill:#87CEEB,stroke:#333,stroke-width:2px
    style M1A fill:#FFB6C1,stroke:#333,stroke-width:2px
    style M1B fill:#FFB6C1,stroke:#333,stroke-width:2px
    style L1A fill:#DDA0DD,stroke:#333,stroke-width:2px
    style L1B fill:#DDA0DD,stroke:#333,stroke-width:2px
    style L1C fill:#DDA0DD,stroke:#333,stroke-width:2px
'''

display(HTML("""
<div style="border:3px solid #2196F3; padding:20px; border-radius:15px; background-color:#E3F2FD;">
    <h3 style="color:#1976D2;">📊 文本分類任務選型決策樹</h3>
    <p><strong>決策邏輯:</strong></p>
    <ul style="line-height:1.8;">
        <li>🟢 <strong>極少數據 (<500):</strong> 零樣本/少樣本學習 (GPT Prompt)</li>
        <li>🔵 <strong>少量數據 (500-5K):</strong> BERT Base 微調 + 數據增強</li>
        <li>🔴 <strong>中等數據 (5K-50K):</strong> TF-IDF (基礎需求) 或 BERT (高效能)</li>
        <li>🟣 <strong>充足數據 (>50K):</strong> 根據資源選擇 BERT Base/Large 或 GPT</li>
    </ul>
</div>
"""))

print("\n🎯 文本分類選型建議:")
print("  1. 數據<500: GPT-3/4 零樣本學習 (快速驗證想法)")
print("  2. 數據 500-5K: BERT Base 微調 (性價比最高)")
print("  3. 數據 5K-50K: TF-IDF+LR (快速) 或 BERT (高精度)")
print("  4. 數據>50K: BERT Large 或從頭訓練 (追求極致效能)")

### 2.2 文本分類方案對比

In [ ]:
# 文本分類不同方案對比
classification_solutions = {
    '方案': ['TF-IDF + LR', 'FastText', 'BERT Base 微調', 'BERT Large 微調', 'GPT-3 零樣本', 'GPT-3 微調'],
    '數據需求': ['5K+', '10K+', '1K+', '5K+', '<100', '1K+'],
    '訓練時間': ['< 1 分鐘', '< 5 分鐘', '30-60 分鐘', '2-4 小時', '0 (無訓練)', '1-2 小時'],
    '推理速度': ['極快 (ms)', '極快 (ms)', '快 (10-50ms)', '中 (50-100ms)', '慢 (1-3s)', '慢 (1-3s)'],
    '準確率 (%)': ['85-90', '88-92', '92-95', '94-97', '80-90', '93-96'],
    '資源需求': ['CPU', 'CPU/GPU', '單 GPU', '單/多 GPU', 'API', 'API/多 GPU'],
    '適用場景': ['快速原型', '大規模部署', '通用方案', '高精度需求', '快速驗證', '高精度+靈活']
}

df_class = pd.DataFrame(classification_solutions)

# 美化表格
def color_by_performance(val):
    if '94-97' in str(val) or '93-96' in str(val):
        return 'background-color: lightgreen; font-weight: bold'
    elif '92-95' in str(val) or '88-92' in str(val):
        return 'background-color: lightyellow'
    return ''

styled_class = df_class.style.applymap(color_by_performance, subset=['準確率 (%)'])

display(HTML("<h3>📋 文本分類方案全面對比</h3>"))
display(styled_class.hide(axis='index').set_properties(**{'text-align': 'center', 'font-size': '10pt'}))

print("\n💡 方案選擇建議:")
print("  - 快速原型/成本敏感: TF-IDF + LR")
print("  - 大規模部署: FastText (速度+效能平衡)")
print("  - 通用推薦: BERT Base 微調 (性價比最高)")
print("  - 高精度需求: BERT Large 微調")
print("  - 快速驗證/極少數據: GPT-3 零樣本")
print("  - 靈活性+高精度: GPT-3 微調")

<a id="3"></a>
## 3. 命名實體識別 (NER) 選型決策樹

### 3.1 NER 任務特點

**任務定義:** 從文本中識別並分類實體 (人名、地名、機構名等)

**常見實體類型:**
- 人名 (PER)
- 地名 (LOC)
- 機構名 (ORG)
- 時間 (TIME)
- 金額 (MONEY)

**評估指標:**
- Entity-level F1-Score
- Token-level Accuracy
- Precision, Recall

**技術特點:**
- 序列標註任務 (BIO 標註)
- 上下文依賴強
- 需要處理實體邊界

---

In [ ]:
# NER 任務選型決策樹
ner_decision_tree = '''
graph TD
    Start[NER 任務] --> Q1{領域?}
    
    Q1 -->|通用領域| General[通用 NER]
    Q1 -->|專業領域| Domain[領域 NER<br/>醫療/金融/法律]
    
    General --> G1{數據量?}
    G1 -->|<1K| G1A[零樣本<br/>spaCy/Flair]
    G1 -->|1K-10K| G1B[BERT 微調<br/>SpanBERT]
    G1 -->|>10K| G1C[BERT Large<br/>或 RoBERTa]
    
    Domain --> D1{有標註數據?}
    D1 -->|無| D1A[弱監督學習<br/>遠程監督]
    D1 -->|少量<1K| D1B[數據增強<br/>+ BERT 微調]
    D1 -->|充足>5K| D1C[領域 BERT<br/>BioBERT/FinBERT]
    
    G1A --> Final1[spaCy<br/>預訓練模型]
    G1B --> Final2[BERT-NER<br/>微調]
    G1C --> Final3[BERT Large<br/>或 RoBERTa-NER]
    D1A --> Final4[遠程監督<br/>+ 自訓練]
    D1B --> Final5[數據增強<br/>+ BERT]
    D1C --> Final6[領域預訓練<br/>模型微調]
    
    style Start fill:#FFD700,stroke:#333,stroke-width:3px
    style Final1 fill:#90EE90,stroke:#333,stroke-width:2px
    style Final2 fill:#90EE90,stroke:#333,stroke-width:2px
    style Final3 fill:#90EE90,stroke:#333,stroke-width:2px
    style Final4 fill:#87CEEB,stroke:#333,stroke-width:2px
    style Final5 fill:#87CEEB,stroke:#333,stroke-width:2px
    style Final6 fill:#87CEEB,stroke:#333,stroke-width:2px
'''

display(HTML("""
<div style="border:3px solid #FF9800; padding:20px; border-radius:15px; background-color:#FFF3E0;">
    <h3 style="color:#E65100;">🏷️ NER 任務選型決策樹</h3>
    <p><strong>關鍵決策點:</strong></p>
    <ol style="line-height:1.8;">
        <li><strong>領域判斷:</strong> 通用 vs 專業領域 (醫療/金融/法律)</li>
        <li><strong>通用領域:</strong> 根據數據量選擇 spaCy/BERT Base/BERT Large</li>
        <li><strong>專業領域:</strong> 優先使用領域預訓練模型 (BioBERT/FinBERT)</li>
        <li><strong>無標註數據:</strong> 遠程監督、弱監督學習</li>
    </ol>
</div>
"""))

print("\n🎯 NER 選型建議:")
print("  1. 通用領域 + 少量數據: BERT Base 微調 (CoNLL-2003 預訓練)")
print("  2. 醫療領域: BioBERT, SciBERT (生物醫學預訓練)")
print("  3. 金融領域: FinBERT (金融新聞預訓練)")
print("  4. 中文 NER: BERT-Chinese, RoBERTa-Chinese")
print("  5. 快速部署: spaCy 預訓練模型 (開箱即用)")

### 3.2 NER 方案對比

In [ ]:
# NER 方案對比
ner_solutions = {
    '方案': ['CRF', 'BiLSTM-CRF', 'spaCy', 'BERT-NER', 'BioBERT', 'SpanBERT'],
    '技術基礎': ['統計模型', 'RNN+CRF', 'CNN+規則', 'Transformer', '領域Transformer', 'Span預測'],
    '數據需求': ['10K+', '5K+', '0 (預訓練)', '1K+', '2K+ (領域)', '2K+'],
    '訓練時間': ['< 10 分鐘', '30-60 分鐘', '0', '1-2 小時', '1-3 小時', '2-4 小時'],
    'F1-Score (%)': ['75-82', '82-88', '85-90', '90-93', '92-95 (領域)', '91-94'],
    '優勢': ['快速訓練', '處理序列', '開箱即用', '通用性強', '領域專精', '邊界準確'],
    '劣勢': ['特徵工程', '訓練慢', '難以自定義', '需要數據', '領域受限', '計算量大']
}

df_ner = pd.DataFrame(ner_solutions)

# 視覺化 F1-Score 對比
fig, ax = plt.subplots(figsize=(12, 6))

f1_scores = [78.5, 85, 87.5, 91.5, 93.5, 92.5]
colors_bars = ['coral', 'skyblue', 'lightgreen', 'gold', 'plum', 'pink']

bars = ax.bar(df_ner['方案'], f1_scores, color=colors_bars, alpha=0.8, edgecolor='black', linewidth=2)

# 添加數值標籤
for bar, score in zip(bars, f1_scores):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 1,
            f'{score:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')

ax.set_ylabel('F1-Score (%)', fontsize=12, fontweight='bold')
ax.set_title('NER 不同方案效能對比', fontsize=15, fontweight='bold')
ax.set_ylim(0, 100)
ax.axhline(y=90, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='90% 基準線')
ax.legend(fontsize=10)
ax.grid(axis='y', alpha=0.3)

plt.xticks(rotation=15, ha='right')
plt.tight_layout()
plt.show()

display(HTML("<h3>📊 NER 方案詳細對比</h3>"))
display(df_ner.style.hide(axis='index').set_properties(**{'text-align': 'center', 'font-size': '10pt'}))

print("\n💡 實戰建議:")
print("  - 快速原型: spaCy 預訓練模型")
print("  - 通用 NER: BERT-NER 微調")
print("  - 醫療領域: BioBERT (F1 可達 93%+)")
print("  - 高精度需求: SpanBERT (邊界識別更準確)")

<a id="4"></a>
## 4. 文本生成任務選型決策樹

### 4.1 文本生成任務類型

文本生成任務可分為多種類型:

#### 1. 機器翻譯
- **任務:** 將源語言翻譯為目標語言
- **評估:** BLEU, METEOR, chrF
- **推薦:** mT5, mBART, NLLB

#### 2. 文本摘要
- **任務:** 生成簡潔摘要
- **類型:** 抽取式 vs 生成式
- **推薦:** BART, PEGASUS, T5

#### 3. 對話生成
- **任務:** 生成自然對話回復
- **評估:** BLEU, Perplexity, 人工評估
- **推薦:** GPT-3/4, DialoGPT, Blenderbot

#### 4. 創意寫作
- **任務:** 故事、詩歌、文章創作
- **評估:** 人工評估、多樣性指標
- **推薦:** GPT-3/4, Claude

---

In [ ]:
# 文本生成任務選型決策樹
generation_tree = '''
graph TD
    Start[文本生成任務] --> Q1{任務類型?}
    
    Q1 -->|機器翻譯| Translation[翻譯任務]
    Q1 -->|文本摘要| Summarization[摘要任務]
    Q1 -->|對話生成| Dialogue[對話任務]
    Q1 -->|創意寫作| Creative[創作任務]
    
    Translation --> T1{語言對?}
    T1 -->|英中/中英| T1A[mT5<br/>mBART]
    T1 -->|多語言| T1B[NLLB<br/>mT5-XXL]
    T1 -->|低資源語言| T1C[mBART50<br/>遷移學習]
    
    Summarization --> S1{摘要類型?}
    S1 -->|抽取式| S1A[BERT<br/>TextRank]
    S1 -->|生成式| S1B[BART<br/>PEGASUS]
    S1 -->|混合| S1C[T5<br/>Longformer]
    
    Dialogue --> D1{對話場景?}
    D1 -->|任務導向| D1A[T5<br/>BERT+生成]
    D1 -->|開放域| D1B[GPT-3/4<br/>DialoGPT]
    D1 -->|多輪對話| D1C[Blenderbot<br/>ChatGPT]
    
    Creative --> C1{創作類型?}
    C1 -->|故事/小說| C1A[GPT-3/4<br/>Claude]
    C1 -->|詩歌/歌詞| C1B[GPT-3<br/>專用模型]
    C1 -->|技術文檔| C1C[GPT-4<br/>Claude 2]
    
    style Start fill:#FFD700,stroke:#333,stroke-width:3px
    style T1A fill:#90EE90,stroke:#333,stroke-width:2px
    style T1B fill:#90EE90,stroke:#333,stroke-width:2px
    style T1C fill:#90EE90,stroke:#333,stroke-width:2px
    style S1A fill:#87CEEB,stroke:#333,stroke-width:2px
    style S1B fill:#87CEEB,stroke:#333,stroke-width:2px
    style S1C fill:#87CEEB,stroke:#333,stroke-width:2px
    style D1A fill:#FFB6C1,stroke:#333,stroke-width:2px
    style D1B fill:#FFB6C1,stroke:#333,stroke-width:2px
    style D1C fill:#FFB6C1,stroke:#333,stroke-width:2px
    style C1A fill:#DDA0DD,stroke:#333,stroke-width:2px
    style C1B fill:#DDA0DD,stroke:#333,stroke-width:2px
    style C1C fill:#DDA0DD,stroke:#333,stroke-width:2px
'''

display(HTML("""
<div style="border:3px solid #9C27B0; padding:20px; border-radius:15px; background-color:#F3E5F5;">
    <h3 style="color:#6A1B9A;">✍️ 文本生成任務選型決策樹</h3>
    <p><strong>生成任務選型關鍵:</strong></p>
    <ul style="line-height:1.8;">
        <li>🟢 <strong>機器翻譯:</strong> mT5 (通用), NLLB (多語言), mBART (低資源)</li>
        <li>🔵 <strong>文本摘要:</strong> BERT (抽取式), BART/PEGASUS (生成式), T5 (混合)</li>
        <li>🔴 <strong>對話生成:</strong> T5 (任務導向), GPT-3/4 (開放域), Blenderbot (多輪)</li>
        <li>🟣 <strong>創意寫作:</strong> GPT-3/4, Claude (通用創作), 專用模型 (詩歌)</li>
    </ul>
</div>
"""))

print("\n🎯 文本生成選型建議:")
print("  1. 機器翻譯: mT5 (多語言通用), NLLB (200+ 語言)")
print("  2. 文本摘要: PEGASUS (專為摘要設計), BART (通用生成)")
print("  3. 對話生成: GPT-3/4 (開放域), DialoGPT (預訓練對話模型)")
print("  4. 創意寫作: GPT-4 (最強), Claude (安全性高)")

### 4.2 生成任務方案對比

In [ ]:
# 文本生成方案對比
generation_solutions = {
    '任務類型': ['機器翻譯', '機器翻譯', '文本摘要', '文本摘要', '對話生成', '對話生成', '創意寫作'],
    '推薦方案': ['mT5', 'NLLB', 'BART', 'PEGASUS', 'DialoGPT', 'GPT-3/4', 'GPT-4'],
    '模型大小': ['580M-13B', '600M-54B', '400M', '568M', '345M', '175B-1T', '1T+'],
    'BLEU/得分': ['35-40', '40-45', '45-50', '48-52', '-', '-', '-'],
    '訓練成本': ['中', '高', '中', '中', '中', '極高', '極高'],
    '推理速度': ['快', '中', '快', '快', '快', '慢', '慢'],
    '特點': ['多語言', '200+語言', '通用生成', '摘要專用', '對話預訓練', '零樣本', '創作能力強']
}

df_gen = pd.DataFrame(generation_solutions)

display(HTML("<h3>📋 文本生成方案對比</h3>"))
display(df_gen.style.hide(axis='index').set_properties(**{'text-align': 'center', 'font-size': '10pt'}))

# 視覺化模型大小與效能關係
fig, ax = plt.subplots(figsize=(12, 7))

model_sizes = [580, 600, 400, 568, 345, 175000, 1000000]  # 單位: 百萬參數
performance = [37.5, 42.5, 47.5, 50, 85, 90, 95]  # 模擬綜合效能
labels = df_gen['推薦方案'].tolist()

scatter = ax.scatter(model_sizes, performance, s=300, alpha=0.6, 
                     c=range(len(labels)), cmap='viridis', edgecolors='black', linewidth=2)

for i, label in enumerate(labels):
    ax.annotate(label, (model_sizes[i], performance[i]), 
                textcoords="offset points", xytext=(0,10), ha='center', fontsize=10, fontweight='bold')

ax.set_xscale('log')
ax.set_xlabel('模型參數量 (百萬)', fontsize=12, fontweight='bold')
ax.set_ylabel('綜合效能評分', fontsize=12, fontweight='bold')
ax.set_title('文本生成模型: 參數規模 vs 效能', fontsize=15, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 生成任務關鍵洞察:")
print("  1. 模型越大,生成質量越高 (但非線性增長)")
print("  2. 專用模型 (PEGASUS) 在特定任務上優於通用大模型")
print("  3. GPT-3/4 適合零樣本場景,但成本高")
print("  4. 中小型模型 (BART/mT5) 微調後性價比高")

<a id="5"></a>
## 5. 問答系統選型決策樹

### 5.1 問答系統分類

#### 1. 抽取式問答 (Extractive QA)
- **任務:** 從給定文本中抽取答案片段
- **代表:** SQuAD, CMRC 2018
- **推薦:** BERT, RoBERTa, ALBERT

#### 2. 生成式問答 (Generative QA)
- **任務:** 生成自然語言答案
- **代表:** MS MARCO, Natural Questions
- **推薦:** T5, BART, GPT-3

#### 3. 檢索式問答 (Retrieval-based QA)
- **任務:** 從文檔庫檢索相關答案
- **代表:** DPR, ColBERT
- **推薦:** Dense Retrieval + Reader

#### 4. 知識庫問答 (KB-QA)
- **任務:** 基於知識圖譜回答問題
- **代表:** WebQuestions, ComplexWebQuestions
- **推薦:** KBQA 專用模型

---

In [ ]:
# 問答系統選型決策樹
qa_tree = '''
graph TD
    Start[問答系統] --> Q1{QA 類型?}
    
    Q1 -->|抽取式| Extractive[從文本抽取答案]
    Q1 -->|生成式| Generative[生成自然語言答案]
    Q1 -->|檢索式| Retrieval[從文檔庫檢索]
    Q1 -->|知識庫| KB[基於知識圖譜]
    
    Extractive --> E1{數據量?}
    E1 -->|<5K| E1A[BERT Base<br/>微調]
    E1 -->|5K-50K| E1B[RoBERTa<br/>ALBERT]
    E1 -->|>50K| E1C[BERT Large<br/>ELECTRA]
    
    Generative --> G1{場景?}
    G1 -->|單文檔| G1A[T5<br/>BART]
    G1 -->|多文檔| G1B[FiD<br/>RAG]
    G1 -->|開放域| G1C[GPT-3/4<br/>ChatGPT]
    
    Retrieval --> R1{文檔規模?}
    R1 -->|小<10K| R1A[BM25<br/>+ BERT Reader]
    R1 -->|中 10K-1M| R1B[DPR<br/>+ FiD]
    R1 -->|大>1M| R1C[ColBERT<br/>+ GPT Reader]
    
    KB --> K1{知識庫?}
    K1 -->|結構化| K1A[KBQA<br/>SPARQL生成]
    K1 -->|半結構化| K1B[混合檢索<br/>+ 生成]
    K1 -->|非結構化| K1C[RAG<br/>GPT-3/4]
    
    style Start fill:#FFD700,stroke:#333,stroke-width:3px
'''

display(HTML("""
<div style="border:3px solid #00BCD4; padding:20px; border-radius:15px; background-color:#E0F7FA;">
    <h3 style="color:#006064;">❓ 問答系統選型決策樹</h3>
    <p><strong>四大 QA 類型選型:</strong></p>
    <ul style="line-height:1.8;">
        <li><strong>抽取式 QA:</strong> BERT (基礎), RoBERTa (進階), ELECTRA (高精度)</li>
        <li><strong>生成式 QA:</strong> T5 (單文檔), FiD/RAG (多文檔), GPT-3/4 (開放域)</li>
        <li><strong>檢索式 QA:</strong> BM25+BERT (小規模), DPR (中規模), ColBERT (大規模)</li>
        <li><strong>知識庫 QA:</strong> KBQA (結構化), 混合方法 (半結構化), RAG (非結構化)</li>
    </ul>
</div>
"""))

print("\n🎯 問答系統選型建議:")
print("  1. 抽取式 QA (SQuAD類): BERT/RoBERTa 微調 (F1: 88-93%)")
print("  2. 生成式 QA: T5 (可控性好), GPT-3 (開放域強)")
print("  3. 檢索式 QA: DPR + FiD (平衡方案)")
print("  4. 知識庫 QA: RAG (檢索增強生成) + GPT-3/4")

### 5.2 問答系統方案對比

In [ ]:
# 問答系統方案對比
qa_solutions = {
    'QA 類型': ['抽取式', '抽取式', '生成式', '生成式', '檢索式', '檢索式', '知識庫'],
    '方案': ['BERT-QA', 'RoBERTa-QA', 'T5-QA', 'GPT-3 QA', 'DPR+FiD', 'ColBERT', 'RAG'],
    'F1/EM (%)': ['88/81', '91/85', '-', '-', '42 (EM)', '45 (EM)', '-'],
    '優勢': ['簡單高效', '高準確率', '靈活生成', '零樣本', '可擴展', '高效檢索', '知識更新快'],
    '劣勢': ['需答案在文本', '訓練慢', '需大量數據', '成本高', '兩階段複雜', '計算量大', '檢索質量依賴'],
    '適用場景': ['單文檔閱讀理解', '高精度需求', '多樣化答案', '快速原型', '大規模文檔', '億級檢索', '動態知識']
}

df_qa = pd.DataFrame(qa_solutions)

display(HTML("<h3>📋 問答系統方案全面對比</h3>"))
display(df_qa.style.hide(axis='index').set_properties(**{'text-align': 'center', 'font-size': '10pt'}))

# 視覺化不同 QA 類型的效能
fig, ax = plt.subplots(figsize=(10, 6))

qa_types = ['抽取式\nBERT', '抽取式\nRoBERTa', '檢索式\nDPR+FiD', '檢索式\nColBERT']
em_scores = [81, 85, 42, 45]
colors_qa = ['skyblue', 'lightgreen', 'coral', 'plum']

bars = ax.bar(qa_types, em_scores, color=colors_qa, alpha=0.8, edgecolor='black', linewidth=2)

for bar, score in zip(bars, em_scores):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 2,
            f'{score}%', ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('Exact Match (%)', fontsize=12, fontweight='bold')
ax.set_title('不同 QA 方案效能對比 (SQuAD/Natural Questions)', fontsize=14, fontweight='bold')
ax.set_ylim(0, 100)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 QA 系統選型關鍵:")
print("  1. 答案在給定文本: 抽取式 QA (BERT/RoBERTa)")
print("  2. 需生成完整答案: 生成式 QA (T5/GPT-3)")
print("  3. 大規模文檔檢索: 檢索式 QA (DPR+FiD)")
print("  4. 知識動態更新: RAG (檢索增強生成)")

<a id="6"></a>
## 6. 模型大小與效能權衡

### 6.1 模型規模分類

根據參數量,可將模型分為:

#### 1. 微型模型 (Tiny: <50M)
- **代表:** DistilBERT (66M), TinyBERT (14.5M)
- **優勢:** 極快推理,可部署在移動端
- **劣勢:** 效能損失 5-10%
- **適用:** 邊緣設備,實時應用

#### 2. 小型模型 (Small: 50M-200M)
- **代表:** BERT-Base (110M), RoBERTa-Base (125M)
- **優勢:** 效能與速度平衡
- **劣勢:** 複雜任務表現有限
- **適用:** 大多數 NLP 任務

#### 3. 中型模型 (Medium: 200M-1B)
- **代表:** BERT-Large (340M), GPT-2 (1.5B)
- **優勢:** 高效能,可處理複雜任務
- **劣勢:** 推理較慢,需 GPU
- **適用:** 高精度需求

#### 4. 大型模型 (Large: 1B-100B)
- **代表:** GPT-3 (175B), PaLM (540B)
- **優勢:** 零樣本學習,涌現能力
- **劣勢:** 極高成本,推理慢
- **適用:** 通用智能,研究

#### 5. 超大型模型 (XL: >100B)
- **代表:** GPT-4 (1T+), PaLM 2 (340B+)
- **優勢:** 最強能力,多模態
- **劣勢:** 僅 API 可用,成本極高
- **適用:** 尖端應用,對話系統

---

In [ ]:
# 模型規模與效能權衡分析
model_scale_data = {
    '模型': ['TinyBERT', 'DistilBERT', 'BERT-Base', 'BERT-Large', 'GPT-2', 'GPT-3', 'GPT-4'],
    '參數量 (M)': [14.5, 66, 110, 340, 1500, 175000, 1000000],
    '相對效能 (%)': [85, 92, 100, 105, 110, 125, 135],
    '推理速度 (相對)': [10, 5, 1, 0.3, 0.2, 0.05, 0.02],
    '訓練成本 ($)': [100, 500, 2000, 10000, 50000, 10000000, 100000000],
    '部署難度': ['極易', '易', '中', '中', '難', '極難', '極難']
}

df_scale = pd.DataFrame(model_scale_data)

# 繪製參數量 vs 效能 vs 速度
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# 左圖: 參數量 vs 效能
ax1.plot(df_scale['參數量 (M)'], df_scale['相對效能 (%)'], 'o-', linewidth=2, markersize=10, color='royalblue')
for i, model in enumerate(df_scale['模型']):
    ax1.annotate(model, (df_scale['參數量 (M)'][i], df_scale['相對效能 (%)'][i]),
                textcoords="offset points", xytext=(0,8), ha='center', fontsize=9, fontweight='bold')

ax1.set_xscale('log')
ax1.set_xlabel('參數量 (百萬)', fontsize=12, fontweight='bold')
ax1.set_ylabel('相對效能 (%)', fontsize=12, fontweight='bold')
ax1.set_title('模型規模 vs 效能', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.axhline(y=100, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='BERT-Base 基準')
ax1.legend()

# 右圖: 參數量 vs 推理速度
ax2.plot(df_scale['參數量 (M)'], df_scale['推理速度 (相對)'], 's-', linewidth=2, markersize=10, color='darkorange')
for i, model in enumerate(df_scale['模型']):
    ax2.annotate(model, (df_scale['參數量 (M)'][i], df_scale['推理速度 (相對)'][i]),
                textcoords="offset points", xytext=(0,0.3), ha='center', fontsize=9, fontweight='bold')

ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel('參數量 (百萬)', fontsize=12, fontweight='bold')
ax2.set_ylabel('推理速度 (相對,越大越快)', fontsize=12, fontweight='bold')
ax2.set_title('模型規模 vs 推理速度', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=1, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='BERT-Base 基準')
ax2.legend()

plt.tight_layout()
plt.show()

print("\n📊 模型規模權衡洞察:")
print("  1. 效能增長: 參數量增加 → 效能提升 (邊際遞減)")
print("  2. 速度下降: 參數量增加 → 推理速度大幅下降")
print("  3. 成本飆升: GPT-3 訓練成本 > $10M, GPT-4 > $100M")
print("  4. 甜蜜點: BERT-Base/DistilBERT (效能與速度平衡)")

### 6.2 模型壓縮技術

當需要在資源受限環境部署大模型時,可使用模型壓縮技術:

In [ ]:
# 模型壓縮技術對比
compression_methods = {
    '壓縮技術': ['知識蒸餾\n(Distillation)', '量化\n(Quantization)', '剪枝\n(Pruning)', 
                '低秩分解\n(Factorization)', '混合精度\n(Mixed Precision)'],
    '原理': ['大模型教小模型', '降低數值精度', '移除冗餘參數', '矩陣分解降維', 'FP16/INT8 訓練'],
    '壓縮率': ['2-4x', '2-8x', '2-10x', '1.5-3x', '2x'],
    '效能損失 (%)': ['2-5', '1-3', '3-8', '2-4', '0.5-2'],
    '速度提升': ['2-3x', '2-4x', '2-5x', '1.5-2x', '2x'],
    '代表方法': ['DistilBERT', 'BERT-INT8', 'Lottery Ticket', 'ALBERT', 'NVIDIA Apex']
}

df_compress = pd.DataFrame(compression_methods)

display(HTML("<h3>🔧 模型壓縮技術對比</h3>"))
display(df_compress.style.hide(axis='index').set_properties(**{'text-align': 'center', 'font-size': '10pt'}))

# 視覺化壓縮率 vs 效能損失
fig, ax = plt.subplots(figsize=(10, 6))

compression_rates = [3, 5, 6, 2, 2]
performance_loss = [3.5, 2, 5.5, 3, 1.25]
methods_short = ['蒸餾', '量化', '剪枝', '低秩分解', '混合精度']

scatter = ax.scatter(compression_rates, performance_loss, s=400, alpha=0.6,
                     c=range(len(methods_short)), cmap='Set2', edgecolors='black', linewidth=2)

for i, method in enumerate(methods_short):
    ax.annotate(method, (compression_rates[i], performance_loss[i]),
                textcoords="offset points", xytext=(0,-15), ha='center', fontsize=11, fontweight='bold')

ax.set_xlabel('壓縮率 (倍數)', fontsize=12, fontweight='bold')
ax.set_ylabel('效能損失 (%)', fontsize=12, fontweight='bold')
ax.set_title('模型壓縮技術: 壓縮率 vs 效能損失', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# 理想區域 (高壓縮+低損失)
ax.axhline(y=3, color='green', linestyle='--', linewidth=1.5, alpha=0.5, label='可接受效能損失線')
ax.axvline(x=4, color='blue', linestyle='--', linewidth=1.5, alpha=0.5, label='理想壓縮率線')
ax.legend(fontsize=10)

plt.tight_layout()
plt.show()

print("\n💡 壓縮技術選擇建議:")
print("  1. 追求極致速度: 量化 (INT8) + 剪枝 (最高 10x 壓縮)")
print("  2. 平衡方案: 知識蒸餾 (DistilBERT, 僅損失 3% 效能)")
print("  3. 最小損失: 混合精度訓練 (FP16, 損失 <2%)")
print("  4. 組合使用: 蒸餾 + 量化 + 剪枝 (可達 20x+ 壓縮)")

<a id="7"></a>
## 7. 實際案例選型分析

### 7.1 案例 1: 電商評論情感分析

In [ ]:
# 案例 1: 電商評論情感分析
case1 = {
    '需求描述': '分析用戶評論情感 (正面/負面/中性)',
    '數據量': '50,000 條標註評論',
    '計算資源': '單塊 GPU (V100)',
    '延遲要求': '推理 < 100ms',
    '準確率要求': '> 90%'
}

display(HTML("""
<div style="border:2px solid #4CAF50; padding:15px; border-radius:10px; background-color:#F1F8E9; margin:10px 0;">
    <h4 style="color:#2E7D32;">📦 案例 1: 電商評論情感分析</h4>
    <p><strong>需求:</strong> 分析用戶評論情感 (正面/負面/中性)</p>
    <p><strong>數據量:</strong> 50,000 條標註評論</p>
    <p><strong>計算資源:</strong> 單塊 GPU (V100)</p>
    <p><strong>延遲要求:</strong> 推理 < 100ms</p>
    <p><strong>準確率要求:</strong> > 90%</p>
</div>
"""))

print("\n🔍 決策流程:")
print("  步驟 1: 任務類型 → 文本分類 (三分類)")
print("  步驟 2: 數據量 50K → 充足數據")
print("  步驟 3: 單 GPU → 可使用 BERT Base/Large")
print("  步驟 4: 延遲 <100ms → 需要模型優化")
print("\n✅ 推薦方案:")
print("  【方案 A】 BERT-Base 微調 + 量化 (INT8)")
print("    - 訓練: 2-3 小時")
print("    - 準確率: 92-94%")
print("    - 推理: 30-50ms (量化後)")
print("  【方案 B】 DistilBERT 微調")
print("    - 訓練: 1-2 小時")
print("    - 準確率: 90-92%")
print("    - 推理: 15-30ms")
print("\n💡 最終選擇: DistilBERT (速度優先,準確率滿足要求)")

### 7.2 案例 2: 醫療文本命名實體識別

In [ ]:
# 案例 2: 醫療文本 NER
display(HTML("""
<div style="border:2px solid #2196F3; padding:15px; border-radius:10px; background-color:#E3F2FD; margin:10px 0;">
    <h4 style="color:#1976D2;">🏥 案例 2: 醫療文本命名實體識別</h4>
    <p><strong>需求:</strong> 從醫療記錄中識別疾病、藥物、症狀</p>
    <p><strong>數據量:</strong> 3,000 條標註文本 (專業領域)</p>
    <p><strong>計算資源:</strong> 多 GPU (4x A100)</p>
    <p><strong>準確率要求:</strong> F1 > 85% (專業領域高標準)</p>
</div>
"""))

print("\n🔍 決策流程:")
print("  步驟 1: 任務類型 → NER (序列標註)")
print("  步驟 2: 領域 → 醫療專業領域")
print("  步驟 3: 數據量 3K → 少量領域數據")
print("  步驟 4: 多 GPU → 可訓練大模型")
print("\n✅ 推薦方案:")
print("  【方案 A】 BioBERT 微調")
print("    - 預訓練: 生物醫學文獻 (PubMed, PMC)")
print("    - 訓練: 2-4 小時")
print("    - F1-Score: 87-91% (領域專精)")
print("  【方案 B】 SciBERT 微調")
print("    - 預訓練: 科學文獻")
print("    - F1-Score: 85-89%")
print("  【方案 C】 數據增強 + BERT-Base")
print("    - 使用遠程監督擴充數據至 10K")
print("    - F1-Score: 82-86%")
print("\n💡 最終選擇: BioBERT (領域預訓練優勢明顯)")

### 7.3 案例 3: 智能客服對話系統

In [ ]:
# 案例 3: 智能客服
display(HTML("""
<div style="border:2px solid #FF9800; padding:15px; border-radius:10px; background-color:#FFF3E0; margin:10px 0;">
    <h4 style="color:#E65100;">💬 案例 3: 智能客服對話系統</h4>
    <p><strong>需求:</strong> 7x24小時自動回答客戶問題</p>
    <p><strong>對話數據:</strong> 10,000 條歷史對話 (FAQ 1,000 條)</p>
    <p><strong>計算資源:</strong> 成本敏感,優先 CPU 部署</p>
    <p><strong>要求:</strong> 快速響應 (<1s), 準確率 >80%</p>
</div>
"""))

print("\n🔍 決策流程:")
print("  步驟 1: 任務類型 → 對話生成 (檢索式 + 生成式)")
print("  步驟 2: 場景 → 任務導向對話 (FAQ)")
print("  步驟 3: 資源 → CPU 部署,成本敏感")
print("  步驟 4: 延遲 <1s → 必須優化")
print("\n✅ 推薦方案:")
print("  【階段 1】 FAQ 檢索 (BM25 + BERT Reranking)")
print("    - BM25 快速召回 Top-20")
print("    - DistilBERT 重排序 Top-3")
print("    - 響應時間: 200-500ms")
print("    - 準確率: 85-90% (FAQ 覆蓋場景)")
print("  【階段 2】 生成式回答 (DialoGPT 微調)")
print("    - 處理 FAQ 外問題")
print("    - 響應時間: 1-2s")
print("    - 準確率: 70-80%")
print("  【架構】 混合式: FAQ 檢索 (主) + 生成式 (輔)")
print("\n💡 最終選擇: BM25 + DistilBERT + DialoGPT 混合架構")

<a id="8"></a>
## 8. 決策樹實戰工具

### 8.1 互動式技術選型工具

In [ ]:
# 互動式技術選型決策工具
def recommend_solution(task_type, data_size, compute_resource, latency_req):
    """
    基於輸入參數推薦技術方案
    
    Args:
        task_type: 任務類型 ('classification', 'ner', 'generation', 'qa')
        data_size: 數據量 ('tiny', 'small', 'medium', 'large')
        compute_resource: 計算資源 ('cpu', 'single_gpu', 'multi_gpu')
        latency_req: 延遲要求 ('realtime', 'near_realtime', 'batch')
    
    Returns:
        推薦方案字典
    """
    recommendations = []
    
    # 文本分類決策樹
    if task_type == 'classification':
        if data_size == 'tiny':
            recommendations.append({
                'solution': 'GPT-3/4 零樣本學習',
                'reason': '數據極少,使用大模型零樣本',
                'expected_acc': '80-90%',
                'cost': '高 (API 調用)'
            })
        elif data_size == 'small':
            if latency_req == 'realtime':
                recommendations.append({
                    'solution': 'DistilBERT 微調',
                    'reason': '數據少,延遲要求高',
                    'expected_acc': '88-92%',
                    'cost': '低'
                })
            else:
                recommendations.append({
                    'solution': 'BERT-Base 微調',
                    'reason': '平衡方案,性價比高',
                    'expected_acc': '92-95%',
                    'cost': '中'
                })
        elif data_size == 'medium':
            if compute_resource == 'cpu':
                recommendations.append({
                    'solution': 'TF-IDF + Logistic Regression',
                    'reason': 'CPU 友好,數據充足',
                    'expected_acc': '85-90%',
                    'cost': '極低'
                })
            else:
                recommendations.append({
                    'solution': 'BERT-Base 微調',
                    'reason': '數據充足,可訓練高質量模型',
                    'expected_acc': '93-96%',
                    'cost': '中'
                })
        else:  # large
            if compute_resource == 'multi_gpu':
                recommendations.append({
                    'solution': 'BERT-Large 微調',
                    'reason': '數據與資源充足,追求極致效能',
                    'expected_acc': '95-97%',
                    'cost': '高'
                })
            else:
                recommendations.append({
                    'solution': 'RoBERTa-Base 微調',
                    'reason': '單 GPU 最優選擇',
                    'expected_acc': '94-96%',
                    'cost': '中'
                })
    
    # NER 決策樹
    elif task_type == 'ner':
        if data_size in ['tiny', 'small']:
            recommendations.append({
                'solution': 'BERT-NER 微調 + 數據增強',
                'reason': 'NER 需要較多數據,建議數據增強',
                'expected_f1': '88-91%',
                'cost': '中'
            })
        else:
            recommendations.append({
                'solution': 'SpanBERT 微調',
                'reason': '數據充足,SpanBERT 邊界識別更準確',
                'expected_f1': '91-94%',
                'cost': '中高'
            })
    
    # 文本生成決策樹
    elif task_type == 'generation':
        if data_size == 'tiny':
            recommendations.append({
                'solution': 'GPT-3/4 Prompt Engineering',
                'reason': '數據少,使用大模型零樣本生成',
                'expected_quality': '高',
                'cost': '高 (API)'
            })
        elif compute_resource == 'multi_gpu':
            recommendations.append({
                'solution': 'T5-Large 微調',
                'reason': '資源充足,T5 生成質量高',
                'expected_quality': '高',
                'cost': '高'
            })
        else:
            recommendations.append({
                'solution': 'BART-Base 微調',
                'reason': '平衡方案,生成質量好',
                'expected_quality': '中高',
                'cost': '中'
            })
    
    # 問答系統決策樹
    elif task_type == 'qa':
        if data_size in ['tiny', 'small']:
            recommendations.append({
                'solution': 'BERT-QA 微調',
                'reason': '抽取式 QA,BERT 表現優秀',
                'expected_em': '85-90%',
                'cost': '中'
            })
        else:
            recommendations.append({
                'solution': 'RoBERTa-QA 微調',
                'reason': '數據充足,RoBERTa 效能更好',
                'expected_em': '88-93%',
                'cost': '中高'
            })
    
    # 如果有延遲要求,額外推薦優化方案
    if latency_req == 'realtime' and recommendations:
        recommendations.append({
            'solution': f"{recommendations[0]['solution']} + 量化 (INT8)",
            'reason': '實時要求高,建議模型量化',
            'speedup': '2-4x',
            'cost': '低 (優化成本)'
        })
    
    return recommendations

# 測試案例
print("🔧 互動式技術選型工具\n")

test_cases = [
    {
        'name': '案例 1: 情感分析',
        'params': ('classification', 'medium', 'single_gpu', 'near_realtime')
    },
    {
        'name': '案例 2: 醫療 NER',
        'params': ('ner', 'small', 'multi_gpu', 'batch')
    },
    {
        'name': '案例 3: 文本摘要',
        'params': ('generation', 'medium', 'single_gpu', 'batch')
    }
]

for case in test_cases:
    print(f"\n{'='*60}")
    print(f"📋 {case['name']}")
    print(f"{'='*60}")
    recommendations = recommend_solution(*case['params'])
    for i, rec in enumerate(recommendations, 1):
        print(f"\n推薦方案 {i}:")
        for key, value in rec.items():
            print(f"  {key}: {value}")

print("\n\n💡 使用方式:")
print("  可根據實際需求調用 recommend_solution() 函數獲得推薦方案")
print("  參數說明:")
print("    - task_type: 'classification', 'ner', 'generation', 'qa'")
print("    - data_size: 'tiny'(<1K), 'small'(1K-10K), 'medium'(10K-100K), 'large'(>100K)")
print("    - compute_resource: 'cpu', 'single_gpu', 'multi_gpu'")
print("    - latency_req: 'realtime'(<100ms), 'near_realtime'(<1s), 'batch'(>1s)")

---

## 📚 本課總結

### 核心要點回顧:

1. **技術選型六大維度:**
   - 任務類型、數據規模、計算資源
   - 效能要求、延遲限制、可解釋性

2. **四大任務決策樹:**
   - **文本分類:** BERT Base (通用), DistilBERT (速度), GPT-3 (零樣本)
   - **NER:** BERT-NER (通用), BioBERT (醫療), SpanBERT (高精度)
   - **文本生成:** T5 (通用), BART (摘要), GPT-3/4 (開放域)
   - **問答系統:** BERT-QA (抽取式), DPR+FiD (檢索式), RAG (知識庫)

3. **模型規模權衡:**
   - 參數量增加 → 效能提升 (邊際遞減)
   - 推理速度下降,成本飆升
   - BERT-Base/DistilBERT 是甜蜜點

4. **模型壓縮技術:**
   - 知識蒸餾、量化、剪枝、低秩分解
   - 可達 2-10x 壓縮,損失 <5% 效能

5. **實戰案例:**
   - 電商情感分析: DistilBERT
   - 醫療 NER: BioBERT
   - 智能客服: BM25 + DistilBERT + DialoGPT

---

## 🎯 下節預告

**CH09-03: 進階學習路徑**

我們將探討:
- NLP 工程師技能樹
- 推薦書籍、課程、論文清單
- 開源專案推薦
- 從初學者到專家的學習路線圖

---

## 📖 延伸閱讀

1. **模型對比:**
   - [Papers with Code - NLP Benchmarks](https://paperswithcode.com/area/natural-language-processing)
   - [Hugging Face Model Hub](https://huggingface.co/models)

2. **模型壓縮:**
   - [DistilBERT Paper](https://arxiv.org/abs/1910.01108)
   - [Model Compression Survey](https://arxiv.org/abs/2006.04884)

3. **技術選型指南:**
   - [Google ML Practitioners Guide](https://developers.google.com/machine-learning/guides)
   - [AWS ML Best Practices](https://aws.amazon.com/machine-learning/)

---

### 🙋 問題討論

有任何問題嗎?歡迎在討論區提問!

---

**課程資訊:**
- **作者:** iSpan NLP Team
- **版本:** v1.0
- **最後更新:** 2025-10-17
- **授權:** MIT License (僅供教學使用)